**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-qhep2vkz
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-qhep2vkz
  Resolved https://github.com/Kaggle/learntools.git to commit 50dffce293eade886869ec092ab1287b62566170
  Preparing metadata (setup.py) ... - done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=267758 sha256=eaa52a513283137d5a8cdecdcf1bfb12aa720a6920ba2e442cdc55c6874d6a03
  Stored in directory: /tmp/pip-ephem-wheel-cache-82a1u5q6/wheels/dd/d7/6b/0fc758f52767fd281d6dceded6757c6cb5bb90ccd2dbb1de9f
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,ff0b96c0cada768361b1c9341e11905254644afb,c7fd753040e0170e38049465089bca99c750f5ed8b3f6c...,2016-09-20 07:15:00+00:00,2016-09-20 07:30:00+00:00,1265,7.2,NaN,NaN,NaN,NaN,...,0.0,0.01,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None
1,cf43e111e3b01ec10187626e6f9887e0cd764eeb,c7fd753040e0170e38049465089bca99c750f5ed8b3f6c...,2016-09-20 07:45:00+00:00,2016-09-20 07:45:00+00:00,470,3.6,NaN,NaN,NaN,NaN,...,0.0,0.01,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None
2,3b7d25f05d3ff9e6866a56ce442d5a8c51c1832f,c7fd753040e0170e38049465089bca99c750f5ed8b3f6c...,2016-09-20 08:15:00+00:00,2016-09-20 09:00:00+00:00,2675,18.4,NaN,NaN,NaN,NaN,...,0.0,0.01,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None
3,cefb2ed97d5992d4b0f299bb6a9b48232cbf22ef,c7fd753040e0170e38049465089bca99c750f5ed8b3f6c...,2016-09-20 09:15:00+00:00,2016-09-20 09:30:00+00:00,780,5.6,NaN,NaN,NaN,NaN,...,0.0,0.01,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None
4,298996c35d36114811a04d86e8c7c374f45ea37f,c7fd753040e0170e38049465089bca99c750f5ed8b3f6c...,2016-09-20 10:00:00+00:00,2016-09-20 10:00:00+00:00,452,1.2,NaN,NaN,NaN,NaN,...,0.0,6.40,Cash,303 Taxi,NaN,NaN,None,NaN,NaN,None


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,trip_date,avg_num_trips
0,2016-03-24,27.285714
1,2016-02-17,83.142857
2,2016-03-27,26.714286
3,2016-02-23,65.571429
4,2016-01-19,80.428571


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
#q_1.hint()
# q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [5]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                        OVER(PARTITION BY pickup_community_area
                            ORDER BY  trip_start_timestamp) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                    """

# Check your answer
q_2.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,11,2013-10-03 06:30:00+00:00,2013-10-03 06:30:00+00:00,1
1,11,2013-10-03 16:15:00+00:00,2013-10-03 16:30:00+00:00,2
2,11,2013-10-03 16:45:00+00:00,2013-10-03 17:00:00+00:00,3
3,11,2013-10-03 17:00:00+00:00,2013-10-03 17:15:00+00:00,4
4,11,2013-10-03 17:15:00+00:00,2013-10-03 17:30:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Lines below will give you a hint or solution code
#q_2.hint()
# q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://i.imgur.com/zzHgmlx.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [7]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,1e3120e8b2c9af9f006e47d546cc8cee6564dd40b15abe...,2013-10-03 06:45:00+00:00,2013-10-03 06:45:00+00:00,-30.0
1,1e3120e8b2c9af9f006e47d546cc8cee6564dd40b15abe...,2013-10-03 16:30:00+00:00,2013-10-03 17:00:00+00:00,420.0
2,c0aa103d4801977abf28043387becdf2dc927120803c22...,2013-10-03 05:30:00+00:00,2013-10-03 06:00:00+00:00,255.0
3,c0aa103d4801977abf28043387becdf2dc927120803c22...,2013-10-03 13:15:00+00:00,2013-10-03 13:30:00+00:00,435.0
4,c0aa103d4801977abf28043387becdf2dc927120803c22...,2013-10-03 17:45:00+00:00,2013-10-03 18:00:00+00:00,165.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
#q_3.hint()
# q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*